In [36]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset

from torchnlp.metrics import get_moses_multi_bleu
from sacrebleu import corpus_bleu, raw_corpus_bleu

import matplotlib.pyplot as plt
# plt.switch_backend('agg')
# import matplotlib.ticker as ticker
import numpy as np
import math
from math import exp

%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from utils_vi import *
from model_new import *
from evaluation_fs import *

In [3]:
BATCH_SIZE = 64
EMB_SIZE = 300
HIDDEN_SIZE = 288
EPOCH = 15

teacher_forcing_ratio = 1
learning_rate = 0.002
print_every = 50
val_every = 500

nowpath = os.path.abspath('.')
zh_en_dataDir = nowpath + '//dataset//iwsltzhen'
vi_en_dataDir = nowpath + '//dataset//iwsltvien'
emb_dataDir = nowpath + '//dataset//embedding'
res_dataDir = nowpath + '//results'
res_dataDir_vi = nowpath + '//results//vi_new_attn'
res_dataDir_zh = nowpath + '//results//zh_new_attn'

MAX_LEN_filter = 45

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PAD_token = 0
UNK_token = 1
SOS_token = 2
EOS_token = 3

train_en = []
zh_en_reader(train_en, vi_en_dataDir + '//train.tok.en')
train_zh = []
zh_en_reader(train_zh, vi_en_dataDir + '//train.tok.vi')
dev_zh = []
zh_en_reader(dev_zh, vi_en_dataDir + '//dev.tok.vi')
dev_en = []
zh_en_reader(dev_en, vi_en_dataDir + '//dev.tok.en')
test_zh = []
zh_en_reader(test_zh, vi_en_dataDir + '//test.tok.vi')
test_en = []
zh_en_reader(test_en, vi_en_dataDir + '//test.tok.en')

data_preprocessor(dev_zh)
data_preprocessor(dev_en)
data_preprocessor(train_en)
data_preprocessor(train_zh)
data_preprocessor(test_zh)
data_preprocessor(test_en)

input_lang, output_lang, pairs = prepareData_zh_en_ensemble('vi', 'en', train_zh, train_en)
input_train, output_train = ConvertSentence2numpy(input_lang, output_lang, pairs, UNK_token, EOS_token, MAX_LEN_filter)
input_test, output_test = ConvertSentence2numpy(input_lang, output_lang, list(zip(test_zh, test_en)), UNK_token, EOS_token)
input_dev, output_dev = ConvertSentence2numpy(input_lang, output_lang, list(zip(dev_zh, dev_en)), UNK_token, EOS_token)

print(len(input_train)/len(train_zh), ' remains after filter.')
pre_zh_emb_matrix = Load_pretrained_emb(100000, input_lang, emb_dataDir)
pre_en_emb_matrix = Load_pretrained_emb(100000, output_lang, emb_dataDir)
print(torch.cuda.memory_allocated())

train_dataset = TransDataset(input_train, output_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)
val_dataset = TransDataset(input_dev, output_dev)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

Counting words...
Read 133317 sentence pairs
Counted words:
vi 35530
en 47862
0.9387850011626424  remains after filter.
Embedding_matrix shape is (100000, 300)
Embedding Vocabualry size is 99982
Size of the pretrained embedding matrix for  vi  is  (35530, 300)
0.34171122994652403 words appear in the pretrained dataset
Embedding_matrix shape is (100000, 300)
Embedding Vocabualry size is 99996
Size of the pretrained embedding matrix for  en  is  (47862, 300)
0.6783669717103339 words appear in the pretrained dataset
865280


In [5]:
device

device(type='cuda')

In [6]:
test_dataset_vi = TransDataset(input_test, output_test)
test_loader_vi = torch.utils.data.DataLoader(dataset=test_dataset_vi,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

In [5]:
train_dataset_val = TransDataset(input_train, output_train)
train_loader_val = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

In [7]:
encoder = EncoderRNN(input_lang.n_words, EMB_SIZE, HIDDEN_SIZE, pre_zh_emb_matrix).to(device)
decoder = BahdanauAttnDecoderRNN(EMB_SIZE, HIDDEN_SIZE, output_lang.n_words, pre_en_emb_matrix).to(device)
print('num_parameters_encoder:', sum(p.numel() for p in encoder.parameters()))
print('num_parameters_decoder:', sum(p.numel() for p in decoder.parameters()))

encoder.load_state_dict(torch.load(res_dataDir_vi + "//vi_tmp_encoder_batch_1207_300_288_10.pth"))
decoder.load_state_dict(torch.load(res_dataDir_vi + "//vi_tmp_attn_decoder_batch_1207_300_288_10.pth"))

num_parameters_encoder: 11678520
num_parameters_decoder: 29449278


In [8]:
criterion = nn.NLLLoss()

In [17]:
predict_sentences_BS10 = evaluate(val_loader, encoder, decoder, output_lang, K_BS=10)
val_Bleu_BS10 = corpus_bleu(predict_sentences_BS10, [dev_en]).score

In [19]:
val_Bleu_BS10

18.1417093296604

In [12]:
predict_sentences_BS15[:10]

['when i was a kid , i thought that the first thing was the best country on the world and i said , &quot; we &apos;ve got nothing to envy . &quot;',
 'i was very proud of my country .',
 'in school , we spent a lot of time to learn about the metals of the metals of the needle ii -- except not to learn a lot about the outside world , except the united states , korea and japan was our enemies .',
 'although i &apos;ve been wondering what the outside world is , but i still think that i &apos;m going to live both my life in the same way , until all of a sudden change change .',
 'when i was seven years ago , i witnessed the first time in my life in my life , but i was still thinking my life was completely normal .',
 'my family wasn &apos;t poor , and i was still hungry .',
 'but in 1995 , my mother brought home a letter from a grandmother from a sisters to her mother .',
 'and she wrote : when you read these lines , both five of your family had no longer ever ever existed , because they h

In [19]:
predict_sentences_noBS[:10]

['when i was a little bit , i thought that the first thing was the best country on the world and i used to sing &quot; we don &apos;t have to envy . &quot; ',
 'i was very proud of my country . ',
 'in school , we spent a lot of time to learn about the kim &apos;s lives of the kim ii -- not learning about the african , except the united states , korea and japan was our enemies . ',
 'although i &apos;ve been wondering , i &apos;ve been wondering how the outside world , but i still think i &apos;m going to live in the same way of being done in the same way , until all of the suddenly change . ',
 'when i was seven years , i saw people in the first time in my life , but i was still thinking my life was completely normal . ',
 'my family wasn &apos;t poor , and i was not going to be hungry . ',
 'but in 1995 , my mother brought home a letter from a woman who was a grandmother to do with her mother . ',
 'and in that one , she read these lines , both five of their life has no longer ever e

In [15]:
switcher = {'ge': lambda x, y: len(x) >= y,
        'le': lambda x, y: len(x) <= y}

In [22]:
for i, x in enumerate([0,1,2,3,4,5][2:], 2):
    print(i, x)

2 2
3 3
4 4
5 5


In [80]:
def sampler(inData, dataLoader, orglang, method = 'ge', thre = 100, start=1000):
    switcher = {
        'ge': lambda x, y: len(x) >= y,
        'le': lambda x, y: len(x) <= y
    }
    reList = []
    reLoList = []
    reOrgList = []
    indList = []
    for i, data in enumerate(inData[start:], start):
        if switcher[method](data, thre):
            reList.append(data)
            indList.append(i)
        if len(reList) > 3:
            break
    for i, data in enumerate(dataLoader):
        if i in indList:
            reLoList.append(data)
            reOrgList.append(orglang[i])
    return(reList, reLoList, reOrgList)

In [82]:
test_en_long, test_loader_vi_long, test_org_vi_long = sampler(test_en, test_loader_vi, test_zh, method = 'ge', thre = 150, start=500)

In [83]:
t_predict_sentences_long_BS4 = evaluate(test_loader_vi_long, encoder, decoder, output_lang, K_BS=4)
t_Bleu_long_BS4_r = raw_corpus_bleu(t_predict_sentences_long_BS4, [test_en_long]).score

In [84]:
t_Bleu_long_BS4_r

17.729864200664572

In [85]:
test_org_vi_long

['Cười Nếu bạn nghĩ đến phim " Avata " nếu bạn nghĩ về việc mọi nó đã lay_động mọi người như thế_nào - - đừng để_ý đến câu_chuyện Pocahontas làm_gì - - Tại_sao lại bị lay_động bởi biểu_tượng đến_thế',
 'Gần đây , tôi đã khảo_sát với hơn 2.000 người Mỹ , và trung_bình số lựa_chọn mà người châu mỹ điển_hình đã làm là khoảng 70 lần trong 1 ngày',
 'Và những nhà khoa_học đã dẫn ra tất_cả những nhiệm_vụ khác_nhau mà những CEO đó tham_gia và họ đã tốn bao nhiều thời_gian để thực_hiện những quyết_định liên qua đén những nhiệm_vụ đó',
 'Khi tôi là sinh_viên tốt_nghiệp từ đại_học Stanford , tôi đã từng vào rất nhiều cửa_hàng tạp_hoá mắc tiền , lúc đó , tôi thật_sự khá giàu ,']

In [86]:
t_predict_sentences_long_BS4

['if you think about film , &quot; if you think about things that have moved people like -- don &apos;t pay attention to the story ?',
 'recently , i &apos;ve been investigating over 2,000 americans , and the average number of choices that the american americans have done is about 70 times a day .',
 'and the scientists have led all sorts of different tasks that ceos have been involved and they &apos;ve spent a lot of time to do these decisions .',
 'when i was a graduate student from stanford university , i used to be a lot of grocery shops , at the time , i was pretty rich .']

In [73]:
## Grammar mistakes, 

In [74]:
test_en_long

['And if you think about &quot; Avatar , &quot; if you think of why people were so touched by it -- never mind the Pocahontas story -- why so touched by the imagery ?',
 'I recently did a survey with over 2,000 Americans , and the average number of choices that the typical American reports making is about 70 in a typical day .',
 'And these scientists simply documented all the various tasks that these CEOs engaged in and how much time they spent engaging in making decisions related to these tasks .',
 'So when I was a graduate student at Stanford University , I used to go to this very , very upscale grocery store ; at least at that time it was truly upscale .']

In [87]:
test_en_short, test_loader_vi_short, test_org_vi_short = sampler(test_en, test_loader_vi, test_zh, method = 'le', thre = 40, start=500)

In [76]:
t_predict_sentences_short_BS4 = evaluate(test_loader_vi_short, encoder, decoder, output_lang, K_BS=4)
t_Bleu_short_BS4 = corpus_bleu(t_predict_sentences_short_BS4, [test_en_short]).score

In [77]:
t_Bleu_short_BS4

27.863110283992324

In [88]:
test_org_vi_short

['Cám_ơn rất nhiều',
 'Bây_giờ , nghĩ về những sự lựa_chọn của chính bạn',
 'Cảm_ơn .',
 "Đây là cửa_hàng với tên Drager ' s"]

In [78]:
t_predict_sentences_short_BS4 

['thank you very much .',
 'now , think about your own choices .',
 'thank you .',
 'this is the store with the name of the &quot; star . &quot;']

In [79]:
test_en_short

['Thank you very much .',
 'Think about your own choices .',
 'Thank you .',
 'It was a store called Draeger &apos;s .']

In [14]:
t_Bleu_BS4

18.04698098099584

In [15]:
t_predict_sentences_BS10 = evaluate(test_loader, encoder, decoder, output_lang, K_BS=10)
t_Bleu_BS10 = corpus_bleu(t_predict_sentences_BS10, [test_en]).score

In [16]:
t_Bleu_BS10

17.87303338870817

In [22]:
predict_sentences = evaluate(val_loader, encoder, decoder, output_lang, K_BS=4)
val_Bleu = corpus_bleu(predict_sentences, [dev_en]).score

In [24]:
predict_sentences[:10]

['when i was a little kid , i thought that the first thing was the best country on the world and i used to sing &quot; we don &apos;t have to envy . &quot;',
 'i was very proud of my country .',
 'in school , we spend a lot of time to learn about the kim &apos;s lives of the kim ii -- i &apos;m not going to learn a lot about the outside world , except the united states , korea and japan is our enemies .',
 'although i &apos;ve been wondering , i &apos;ve been wondering where the outside world is , but i still think that i &apos;m going to live in the first day , until all of a sudden change change .',
 'when i was seven years ago , i witnessed the first time in my life in my life , but i was still thinking my life in this is completely normal .',
 'my family wasn &apos;t poor , and i was still hungry .',
 'but in 1995 , my mother brought home a letter from a grandmother from a grandmother to her mother .',
 'and in that writing : when you read these lines , both the family of her five 

In [23]:
val_Bleu

18.52298297858399

In [13]:
dev_en[:10]

['When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;',
 'And I was very proud .',
 'In school , we spent a lot of time studying the history of Kim Il-Sung , but we never learned much about the outside world , except that America , South Korea , Japan are the enemies .',
 'Although I often wondered about the outside world , I thought I would spend my entire life in North Korea , until everything suddenly changed .',
 'When I was seven years old , I saw my first public execution , but I thought my life in North Korea was normal .',
 'My family was not poor , and myself , I had never experienced hunger .',
 'But one day , in 1995 , my mom brought home a letter from a coworker &apos;s sister .',
 'It read , &quot; When you read this , all five family members will not exist in this world , because we haven &apos;t eaten for the past two weeks .',
 'We are lying on the floor together , and our bodies are s

In [11]:
encoder_1 = EncoderRNN(input_lang.n_words, EMB_SIZE, HIDDEN_SIZE, pre_zh_emb_matrix).to(device)
decoder_1 = BahdanauAttnDecoderRNN(EMB_SIZE, HIDDEN_SIZE, output_lang.n_words, pre_en_emb_matrix).to(device)
print('num_parameters_encoder:', sum(p.numel() for p in encoder.parameters()))
print('num_parameters_decoder:', sum(p.numel() for p in decoder.parameters()))

encoder_1.load_state_dict(torch.load(res_dataDir + "//vi_a_encoder_batch_1205_t1300_384_20_2.pth"))
decoder_1.load_state_dict(torch.load(res_dataDir + "//vi_a_attn_decoder_batch_1205_t1300_384_20_2.pth"))

num_parameters_encoder: 12239544
num_parameters_decoder: 35790654


In [12]:
predict_sentences_1 = evaluate(val_loader, encoder_1, decoder_1, output_lang)
val_Bleu_1 = corpus_bleu(predict_sentences_1, [dev_en]).score

In [13]:
predict_sentences_1[:10]

['when i think the , , i the the the , , , i i , i i i don don don &apos;t &apos;t &apos;t &apos;t don &apos;t &apos;t &apos;t &apos;t &apos;t . . ',
 'i was proud proud proud proud proud . . ',
 'in , , , , many , many to to we , , many many many , , , , , , , , , , , , , , , , , , , , , , , the the . . ',
 'but i i knew thinking thinking , , , , , , , , , i , i , i , i i , i i , i life life life life . . ',
 'when i first first first my life , , life , life , , , i i i i life life life life life life life life . . ',
 'my family had poor and , , and i , never , never never . . ',
 'but , mother mother , mother , mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother a mother mother . . . ',
 'and in this , , , , , because , , , , , , because , , , , because , in in in in in in . . . . ',
 'all of our our our , , , , , , , , , our our our our . . . ',
 'i was shocked shocked . ']

In [14]:
val_Bleu_1

3.55278820416851

In [15]:
from evaluation import *

In [16]:
predict_sentences_2 = evaluate(val_loader, encoder_1, decoder_1, output_lang)
val_Bleu_2 = corpus_bleu(predict_sentences_2, [dev_en]).score

In [18]:
val_Bleu_2

1.1955601883853062

In [32]:
predict_sentences_3 = evaluate([item for i, item in enumerate(train_loader_val) if i < 100], encoder_1, decoder_1, output_lang)
val_Bleu_3 = corpus_bleu(predict_sentences_3, [train_en[:100]]).score

In [29]:
val_Bleu_3 = corpus_bleu(predict_sentences_3, [train_en[:100]]).score

In [30]:
val_Bleu_3 

0.45578048872221205

In [33]:
predict_sentences_3[:10]

['the science science science science science . . ',
 'i want to to you you you you you scientific you you you you the the the the the the on . . . ',
 'there &apos;s like when when when when when when when when when when when when when when when when when when when when when climate and . . ',
 'both are a scientific scientific science science science science . ',
 'the headlines headlines this this , headlines the the , , , , , , , , the the the the the the the the the the the the their . . . ',
 'scientists scientists written by by 40 40 . . . ',
 'they wrote about 1,000 pages about 1,000 . . ',
 'and all all of scientists are from and and and and from from reviewed reviewed from . . ',
 'it &apos;s a big community , community community community , community community community community the the the the the the the the the the . . . ',
 'every year , san san san francisco san francisco francisco . . ']

In [34]:
val_Bleu_3 

1.2053637863838014

In [35]:
from evaluation_fs import *

In [48]:
predict_sentences_4 = evaluate(val_loader, encoder_1, decoder_1, output_lang)
val_Bleu_4 = corpus_bleu(predict_sentences_4, [dev_en]).score
val_Bleu_4

3.1880997579653823

In [49]:
predict_sentences_4[:10]

['when i think , the , i , the the i the i the i i i don don don &apos;t &apos;t &apos;t &apos;t &apos;t &apos;t &apos;t &apos;t &apos;t &apos;t . . &quot; <eos>',
 'i was proud proud proud proud proud . . <eos>',
 'in many , , many many many many of of , , , , to to , , , , , , , , , , , , , , , , , , , , , , , the . . <eos>',
 'but i i knew thinking thinking , , , , , , , , , , , i , i , , i i , , i i i life life life life . . <eos>',
 'when i i my first first , , life , my life life life life , , , , i i life life life life life . . <eos>',
 'my family had poor poor , and poor , i never never never never . . <eos>',
 'but , mother , mother , mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother mother . . <eos>',
 'and in this this , , , , , , because , , , , , because , because because because , because in in in in in . . . . <eos>',
 'all of our our our , , , , , , , , , , our our our . . <eos>',
 'i was shock

In [85]:
predict_sentences = evaluate(val_loader, encoder, decoder, output_lang)

2.0872736788851187e-20
1.885156998846001e-20
1.7544745128033867e-20
6.995059144378444e-21


In [90]:
predict_sentences_6 = evaluate(val_loader, encoder, decoder, output_lang)
val_Bleu_6 = corpus_bleu(predict_sentences_6, [dev_en]).score

In [92]:
predict_sentences_6[:10]

['when i was a little , i i think that the best the the the the the and and i i love we we we we we &quot; &quot;',
 'i was very proud about my country .',
 'in school , we spent a lot of time to learn about the of of the , , , , not not lot about the outside the , , , , , , and and our',
 'even though i asked myself to the the world , but but i i i i life life life life in , , , , all everything <pad>',
 'when i was seven seven , i i see the the the the time first time , , but i my life life life life was <pad>',
 'my family was poor , and i didn &apos;t have to hungry .',
 'but in a day day , , mother mother took a a a of of a a to the to her . <pad>',
 'and it was : : when you read these these family family family family not not , , , because because had no no in in <pad>',
 'all all on the the , our our bodies , that that as as as .',
 'i was shocked .']

In [93]:
val_Bleu_6

9.507029939562269

In [94]:
## Try the best available

In [95]:
encoder_1 = EncoderRNN(input_lang.n_words, EMB_SIZE, HIDDEN_SIZE, pre_zh_emb_matrix).to(device)
decoder_1 = BahdanauAttnDecoderRNN(EMB_SIZE, HIDDEN_SIZE, output_lang.n_words, pre_en_emb_matrix).to(device)
print('num_parameters_encoder:', sum(p.numel() for p in encoder.parameters()))
print('num_parameters_decoder:', sum(p.numel() for p in decoder.parameters()))

encoder_1.load_state_dict(torch.load(res_dataDir + "//vi_tmp_encoder_batch_1206_t1300_256_10.pth"))
decoder_1.load_state_dict(torch.load(res_dataDir + "//vi_tmp_attn_decoder_batch_1206_t1300_256_10.pth"))

num_parameters_encoder: 11516088
num_parameters_decoder: 28072766


In [109]:
predict_sentences_7 = evaluate(val_loader, encoder_1, decoder_1, output_lang, K_BS=15)
val_Bleu_7 = corpus_bleu(predict_sentences_7, [dev_en]).score

/beegfs/fs1520/NLP/project/model.py:129: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.log_softmax(self.out(output))


In [110]:
val_Bleu_7

10.03096524072704

In [112]:
predict_sentences_7[:10]

['when i got a child , i think that the best is the the the the the world and i i love we we we we we',
 'i am very proud about my country .',
 'in school , we spend a lot of time to learn about the the of the the , , , didn &apos;t &apos;t learn about the the , , , , , ,',
 'even though i asked wonder , i didn &apos;t know the world outside , but i still think that i live alive in the world , until all of all of all .',
 'when i was seven seven , i i saw the that the the the time first time time , , but but i my life life <pad>',
 'my family wasn &apos;t poor , and i didn &apos;t have been hungry .',
 'but on a day day , my mother mother took a a a a a a people the with',
 'and it was : : when you read these these , my family family family not not so , , because because &apos;t had &apos;t <pad>',
 'all all on the floor , and our body bodies have to feel like that is dying .',
 'i was shocked .']

In [97]:
predict_sentences_8 = evaluate(val_loader, encoder_1, decoder_1, output_lang, K_BS=6)
val_Bleu_8 = corpus_bleu(predict_sentences_8, [dev_en]).score

In [99]:
val_Bleu_8

9.8166175596524

In [104]:
predict_sentences_7_org = evaluate_org(val_loader, encoder_1, decoder_1, output_lang)
val_Bleu_7_org = corpus_bleu(predict_sentences_7_org, [dev_en]).score

/beegfs/fs1520/NLP/project/model.py:129: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.log_softmax(self.out(output))


In [105]:
val_Bleu_7_org

8.33661623945737

In [107]:
predict_sentences_10 = evaluate(val_loader, encoder_1, decoder_1, output_lang, K_BS=10)
val_Bleu_10 = corpus_bleu(predict_sentences_10, [dev_en]).score

In [108]:
val_Bleu_10

9.951006851903207

In [20]:
from math import exp

In [133]:
def evaluate(loader, encoder, decoder, output_lang, MAX_LENGTH = 100, K_BS=4):
    def Beam_Search_Dec_Iterator(MAX_LENGTH, decoder_input, decoder_hidden, encoder_outputs, K_BS):
    ## K_BS: select K max possible values in candidates
        last_candidates = [[1, [decoder_input], decoder_hidden]]
        out_candidates = []
        min_proba = 1
        for di in range(MAX_LENGTH):
            cur_candidates = []
            if len(last_candidates) <= 0:
                break
            for proba, cur_centense, decoder_hidden in last_candidates:
                decoder_input = cur_centense[-1].detach()
                # print(decoder_input)
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input.to(device), decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.data.topk(K_BS)
                # print([exp(x.item()) for x in topv[0]], topi)
                for j, v in enumerate(topi[0]):
                    # print()
                    this_cen = cur_centense+[v.unsqueeze(0).unsqueeze(0)]
                    # this_proba = proba*(exp(topv[0][j].item()))
                    this_proba = proba*exp(topv[0][j].item())
                    this_hidden = decoder_hidden
                    if di == MAX_LENGTH-1:
                        this_cen += [torch.tensor([[EOS_token]])]
                        out_candidates.append([this_proba, this_cen])
                    if v.item() == EOS_token or v.item() == PAD_token:
                        this_cen += [torch.tensor([[EOS_token]])]
                        out_candidates.append([this_proba, this_cen])
                        # min_proba = min(min_proba, this_proba)
                        # K_BS -= 1
                    else:
                        cur_candidates.append([this_proba, this_cen, this_hidden])
            # print(cur_candidates[0])
            cur_candidates = sorted(cur_candidates, key=lambda x:-x[0])[:K_BS]
            last_candidates = cur_candidates[:K_BS]

        out_candidates = sorted(out_candidates, key=lambda x:-x[0])[:K_BS]
        out_sentences = []
        for _, can in out_candidates:
            # print(_)
            out_sentences.append(' '.join([output_lang.index2word[v.item()] for v in can]).strip('SOS EOS'))

        return(out_sentences[0])

    encoder.eval()
    decoder.eval()
    sentences_list = []

    with torch.no_grad():
        for i, (input_sentence, output_sentence) in enumerate(loader):
            encoder_hidden = encoder.initHidden(input_sentence.size()[0]).cuda()
            encoder_outputs, encoder_hidden = encoder(input_sentence.to(device), encoder_hidden)

            decoder_input = torch.tensor(np.array([[SOS_token]]), device=device)
            decoder_hidden = encoder_hidden
            #predicted_sentence = []
            predicted_sentences = Beam_Search_Dec_Iterator(MAX_LENGTH, decoder_input, decoder_hidden, encoder_outputs, K_BS)

            sentences_list.append(predicted_sentences.strip('<pad>'))

    return sentences_list

In [5]:
## Beam Search

In [10]:
def Beam_Search_Dec_Iterator(MAX_LENGTH, decoder_input, decoder_hidden, encoder_outputs, K_BS):
    ## K_BS: select K max possible values in candidates
    last_candidates = [[1, [decoder_input], decoder_hidden]]
    out_candidates = []
    min_proba = 1
    for di in range(MAX_LENGTH):
        cur_candidates = []
        if len(last_candidates) <= 0:
            break
        for proba, cur_centense, decoder_hidden in last_candidates:
            decoder_input = cur_centense[-1].detach()
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input.to(device), decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(K_BS)
            for j, v in enumerate(topi):
                this_cen = cur_centense+[v]
                this_proba = proba*topv[j].item()
                this_hidden = decoder_hidden
                if di == MAX_LENGTH-1:
                    this_cen += EOS_token
                if v.item() == EOS_token or v.item() == PAD_token:
                    this_cen += EOS_token
                    out_candidates.append([this_proba, this_cen])
                    min_proba = min(min_proba, this_proba)
                    # K_BS -= 1
                else:
                    cur_candidates.append([this_proba, this_cen, this_hidden])
        cur_candidates = sorted(cur_candidates, key=lambda x:-x[0])[:K_BS]
        last_candidates = []
        for cen in cur_candidates:
            if cen[0] > min_proba:
                if len(last_candidates) < K_BS:
                    last_candidates.append(cen)
                else:
                    break
                    
    out_candidates = sorted(out_candidates, key=lambda x:-x[0])[:K_BS]
    out_sentences = []
    for _, can in out_candidates:
        out_sentences.append(' '.join([output_lang.index2word[v.item()] for v in can]))

    return(out_sentences)

In [119]:
'asdfasfsfs<eos>'.strip('<eos>')

'asdfasfsf'

In [103]:
def evaluate_org(loader, encoder, decoder, output_lang, MAX_LENGTH = 100):
    encoder.eval()
    decoder.eval()
    sentences_list = []

    with torch.no_grad():
        for i, (input_sentence, output_sentence) in enumerate(loader):

            encoder_hidden = encoder.initHidden(input_sentence.size()[0]).cuda()
            encoder_outputs, encoder_hidden = encoder(input_sentence.to(device), encoder_hidden)

            decoder_input = torch.tensor(np.array([[SOS_token]]), device=device)
            decoder_hidden = encoder_hidden
            #predicted_sentence = []
            predicted_sentence = ''

            for di in range(MAX_LENGTH):
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input.to(device), decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token or topi.item() == PAD_token:
                    # predicted_sentence += '<eos>'
                    predicted_sentence += ''
                    break
                else:
                    #predicted_sentence.append(output_lang.index2word[topi.item()])
                    predicted_sentence += output_lang.index2word[topi.item()]
                    predicted_sentence += " "

                decoder_input = topi.detach()
                #decoder_input = decoder_input.unsqueeze(0)
            if di==MAX_LENGTH -1:
                # predicted_sentence += '<eos>'
                predicted_sentence += ''

            sentences_list.append(predicted_sentence.strip('SOS EOS'))
            #decoder_attentions_list.append(decoder_attentions[:di + 1])

        #return decoded_words_list, decoder_attentions_list
    return sentences_list


In [102]:
a = 'SOS asdfse EO'
a.strip('SOS eEOS')

'asdfs'

In [ ]:
,'wow wow wow wow i am not shabi','wow wow wow i am so tai niubi']

In [21]:
zoe = torch.Tensor([0.1,0.2,0.5,0.2])

In [25]:
eoe = F.softmax(zoe)

/beegfs/fs1520/pytorch_gpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [28]:
eoe

tensor([ 0.2127,  0.2350,  0.3173,  0.2350])

In [46]:
cce = eoe.data.topk(3)[1]
for i in cce:
    print(i)

tensor(2)
tensor(1)
tensor(3)


In [32]:
sorted([[1,2,3],[2,1,1],[3,0,1]], key=lambda x:-x[0])

[[3, 0, 1], [2, 1, 1], [1, 2, 3]]

In [36]:
aces = []
aces.append('wow')

In [37]:
aces

['wow']

In [38]:
aces.append('wowowow')

In [39]:
aces

['wow', 'wowowow']

In [3]:
ac = {1:1,2:2,3:3}
for i in list(ac.keys()):
    if i != 2:
        ac.pop(i)

In [4]:
ac

{2: 2}

### Chinese_Test

In [95]:
from utils import *
from model_new_simple import *
from evaluation_fs import *

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PAD_token = 0
UNK_token = 1
SOS_token = 2
EOS_token = 3

train_en = []
zh_en_reader(train_en, zh_en_dataDir + '//train.tok.en')
train_zh = []
zh_en_reader(train_zh, zh_en_dataDir + '//train.tok.zh')
dev_zh = []
zh_en_reader(dev_zh, zh_en_dataDir + '//dev.tok.zh')
dev_en = []
zh_en_reader(dev_en, zh_en_dataDir + '//dev.tok.en')
test_zh = []
zh_en_reader(test_zh, zh_en_dataDir + '//test.tok.zh')
test_en = []
zh_en_reader(test_en, zh_en_dataDir + '//test.tok.en')

data_preprocessor(dev_zh)
data_preprocessor(dev_en)
data_preprocessor(train_en)
data_preprocessor(train_zh)
data_preprocessor(test_zh)
data_preprocessor(test_en)

input_lang, output_lang, pairs = prepareData_zh_en_ensemble('zh', 'en', train_zh, train_en)
input_train, output_train = ConvertSentence2numpy(input_lang, output_lang, pairs, UNK_token, EOS_token, MAX_LEN_filter)
input_test, output_test = ConvertSentence2numpy(input_lang, output_lang, list(zip(test_zh, test_en)), UNK_token, EOS_token)
input_dev, output_dev = ConvertSentence2numpy(input_lang, output_lang, list(zip(dev_zh, dev_en)), UNK_token, EOS_token)

print(len(input_train)/len(train_zh), ' remains after filter.')
pre_zh_emb_matrix = Load_pretrained_emb(100000, input_lang, emb_dataDir)
pre_en_emb_matrix = Load_pretrained_emb(100000, output_lang, emb_dataDir)
print(torch.cuda.memory_allocated())

train_dataset = TransDataset(input_train, output_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)
val_dataset = TransDataset(input_dev, output_dev)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

Counting words...
Read 213377 sentence pairs
Counted words:
zh 88426
en 60699
0.9515880343242242  remains after filter.
Embedding_matrix shape is (100000, 300)
Embedding Vocabualry size is 99994
Size of the pretrained embedding matrix for  zh  is  (88426, 300)
0.3373894555899849 words appear in the pretrained dataset
Embedding_matrix shape is (100000, 300)
Embedding Vocabualry size is 99996
Size of the pretrained embedding matrix for  en  is  (60699, 300)
0.6207186279839866 words appear in the pretrained dataset
165764096


In [91]:
test_dataset_zh = TransDataset(input_test, output_test)
test_loader_zh = torch.utils.data.DataLoader(dataset=test_dataset_zh,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

In [19]:
train_dataset_val = TransDataset(input_train, output_train)
train_loader_val = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

In [92]:
EMB_SIZE=300
HIDDEN_SIZE=384

In [93]:
input_lang.n_words

88426

In [96]:
encoder = EncoderRNN(input_lang.n_words, EMB_SIZE, HIDDEN_SIZE, pre_zh_emb_matrix).to(device)
decoder = BahdanauAttnDecoderRNN(EMB_SIZE, HIDDEN_SIZE, output_lang.n_words, pre_en_emb_matrix).to(device)
print('num_parameters_encoder:', sum(p.numel() for p in encoder.parameters()))
print('num_parameters_decoder:', sum(p.numel() for p in decoder.parameters()))

encoder.load_state_dict(torch.load(res_dataDir_zh + "//zh_tmp_encoder_1210_simple_300_384_15.pth"))
decoder.load_state_dict(torch.load(res_dataDir_zh + "//zh_tmp_attn_decoder_1210_simple_300_384_15.pth"))


num_parameters_encoder: 28108344
num_parameters_decoder: 43551423


In [134]:
test_en_long, test_loader_zh_long, test_org_zh_long = sampler(test_en, test_loader_zh, test_zh, method = 'ge', thre = 150, start=600)

In [135]:
zh_predict_sentences_long_BS4 = evaluate(test_loader_zh_long, encoder, decoder, output_lang, K_BS=4)
zh_Bleu_long_BS4 = raw_corpus_bleu(zh_predict_sentences_long_BS4, [test_en_long]).score

In [136]:
zh_Bleu_long_BS4

7.821592547308882

In [137]:
test_org_zh_long

['这 上面 说 本 飞机 有 很多 精密 部件 协作 保障 你 飞行 安全 这 是不是 不是 让 你 很 有 信心',
 '我们 已经 体会 到 作为 一个 老师 你 的话 是 有 影响 影响力 的 如果 你 是 个 很 慈善 的 老师 你 的 教导 就 格外 意味 意味深长 深长',
 '我 记得 当 我 八九 八九岁 九岁 时 的 一次 我 早上 醒来 跑 到 客厅 所有 的 表兄 表兄妹 兄妹 都 在',
 '大概 十五 还是 二十 二十分 二十分钟 十分 十分钟 分钟 之后 她 站起 起来 穿过 过房 房间 牵住 我 的 手 对 我 说 过来 布莱恩 莱恩 我们 得 谈谈']

In [138]:
zh_predict_sentences_long_BS4

['nd it says , &quot; well , ben plane has a lot of sophisticated parts , collaborative security , and you can fly , and you ',
 'nd we &apos;ve experienced that as a teacher , you have a influence , and if you &apos;re a great teacher , you teach it , you ',
 'nd i remember , when i was nine years old , i woke up to the living room and i woke up the living room with my life , and the brother brother ',
 'it &apos;s about 15 minutes later , and after 10 minutes , she stood up through the room , and i said , &quot; brian , we &apos;ve got to ']

In [139]:
test_en_long

['It says , &apos; This plane has hundreds of thousands of tiny parts working together to make you a safe flight . &apos; Doesn &apos;t that make you feel confident ? &quot;',
 'And I think what we &apos;ve learned is that , if you &apos;re a teacher your words can be meaningful , but if you &apos;re a compassionate teacher , they can be especially meaningful .',
 'And I remember , when I was about eight or nine years old , waking up one morning , going into the living room , and all of my cousins were running around .',
 'And after about 15 or 20 minutes of this , she got up and she came across the room and she took me by the hand and she said , &quot; Come on , Bryan . You and I are going to have a talk . &quot;']

In [127]:
test_en_short, test_loader_zh_short, test_org_zh_short = sampler(test_en, test_loader_zh, test_zh, method = 'le', thre = 40, start=600)

In [128]:
zh_predict_sentences_short_BS4 = evaluate(test_loader_zh_short, encoder, decoder, output_lang, K_BS=4)
zh_Bleu_short_BS4 = raw_corpus_bleu(zh_predict_sentences_short_BS4, [test_en_short]).score

In [129]:
zh_Bleu_short_BS4

16.43890828734302

In [130]:
test_org_zh_short

['所以 它 是 自我 回馈 的', '我们 正在 起飞', '我 妈妈 是 十个 个里 最 年轻 的', '我 永远 忘不掉 不掉']

In [131]:
zh_predict_sentences_short_BS4

['so it &apos;s self .',
 'we &apos;re taking off .',
 'my mom was 10 .',
 'i &apos;ll never forget that .']

In [132]:
test_en_short

['So it &apos;s feeding back on itself .',
 'We &apos;re taking off .',
 'My mom was the youngest of her 10 kids .',
 'I never will forget it .']

In [22]:
encoder = EncoderRNN(input_lang.n_words, EMB_SIZE, HIDDEN_SIZE, pre_zh_emb_matrix).to(device)
decoder = BahdanauAttnDecoderRNN(EMB_SIZE, HIDDEN_SIZE, output_lang.n_words, pre_en_emb_matrix).to(device)
print('num_parameters_encoder:', sum(p.numel() for p in encoder.parameters()))
print('num_parameters_decoder:', sum(p.numel() for p in decoder.parameters()))

encoder.load_state_dict(torch.load(res_dataDir_zh + "//zh_tmp_encoder_1210_simple_t2_300_384_15.pth"))
decoder.load_state_dict(torch.load(res_dataDir_zh + "//zh_tmp_attn_decoder_1210_simple_t2_300_384_15.pth"))

num_parameters_encoder: 28108344
num_parameters_decoder: 43551423


In [24]:
zh_predict_sentences_BS4 = evaluate(test_loader, encoder, decoder, output_lang, K_BS=4)
zh_Bleu_BS4 = corpus_bleu(zh_predict_sentences_BS4, [test_en]).score

/beegfs/fs1520/NLP/project/model_new_simple.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.log_softmax(self.out(output))


In [25]:
zh_Bleu_BS4

11.967703355055011

In [26]:
zh_predict_sentences_BS4[:10]

['hi , hi . i &apos;m kevin . i &apos;m going to analyze managers on youtube , and my profession is watching youtube .',
 'this is true .',
 'so today , we &apos;re going to talk about why we &apos;re going <pad>',
 'we all wanted to be a star celebrity singer , and when i was young , it was very tough to do .',
 'but now , the internet now allows it to come true , so we can create the idea of doing things , and then the moment of success became part of the world .',
 'any of you can be famous in the internet , in',
 'but there &apos;s over 48 hours of video uploaded to youtube .',
 'in many times , there &apos;s only a lot of numbers , and you see , people are people and <pad>',
 'how did this happen ?',
 'the three keys were opening up the community , engaged , and really unexpected .']

In [29]:
zh_predict_sentences_BS8 = evaluate(test_loader, encoder, decoder, output_lang, K_BS=8)
zh_Bleu_BS8 = corpus_bleu(zh_predict_sentences_BS8, [test_en]).score

In [32]:
zh_Bleu_BS8

11.80452819442737